In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
# from tensorflow import set_random_seed
# from numpy.random import seed
# tensorflow.random.set_seed(x2)
# seed(1)
import pandas as pd
import numpy as np
import string, os

In [3]:
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(2)
seed(1)

In [5]:
import tensorflow as tf
tf.random.set_seed(2) 
import pandas as pd
import numpy as np
import string, os 

In [6]:
dir = './newyork_headline/'
headlines = []
for filename in os.listdir(dir):
    if 'Articles' in filename:
        print(filename)
        articledata = pd.read_csv(dir + filename)
        headlines.extend(list(articledata.headline.values))
        

headlines = [h for h in headlines if h != "Unknown"]
len(headlines)

ArticlesFeb2017.csv
ArticlesFeb2018.csv
ArticlesJan2017.csv
ArticlesJan2018.csv


3493

In [7]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

data = [clean_text(x) for x in headlines]
data[:10]

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree']

In [8]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(data):
    ## tokenization
    tokenizer.fit_on_texts(data)
    total_words = len(tokenizer.word_index) + 1
    print(total_words)
    
    input_sequences = []
    for line in data:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(data)
inp_sequences[:10]

6547


[[636, 139],
 [636, 139, 140],
 [636, 139, 140, 84],
 [636, 139, 140, 84, 815],
 [636, 139, 140, 84, 815, 292],
 [636, 139, 140, 84, 815, 292, 62],
 [636, 139, 140, 84, 815, 292, 62, 46],
 [636, 139, 140, 84, 815, 292, 62, 46, 129],
 [428, 2539],
 [428, 2539, 2540]]

In [9]:
def generate_padded_sequences(sequences):
    max_sequence_len = max([len(x) for x in sequences])
    sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = sequences[:,:-1],sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [19]:
#model creation
input_len = max_sequence_len - 1
model = Sequential()
model.add(Embedding(total_words, 10, input_length=input_len))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 23, 10)            65470     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6547)              661247    
Total params: 771,117
Trainable params: 771,117
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [14]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [21]:
print(generate_text("united states", 5, model, max_sequence_len))


United States Adversary Miles Miles Miles Teams


In [16]:
print(generate_text("joe biden", 20, model, max_sequence_len))

Joe Biden Arpaios Latest Offense Running For Senate Dies At The Year Of Progress Was The Tax Relatives Tv Vows Was The


In [17]:
print(generate_text("donald trump", 9, model, max_sequence_len))

Donald Trump Middleschool President With The Way Of The Trump But
